In [75]:
import openai as client
import json
# assistant = client.beta.assistants.create(
#     name="Book Assistant",
#     instructions="You help users with their question on the files they upload.",
#     model="gpt-4-1106-preview",
#     tools=[{"type": "retrieval"}],
# )
assistant_id = 'asst_8esEH4JjqkuJZaY2GpqZyA4y'
# assistant.id

In [76]:
thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": "I want you to help me with this file",
        }
    ]
)
thread

Thread(id='thread_nalDBgi0EVwwpHeofMsaaedN', created_at=1713116686, metadata={}, object='thread')

In [77]:
file = client.files.create(
    file=client.file_from_path("../files/chapter_one.txt"), purpose="assistants"
)
file

FileObject(id='file-g9IIL5WN6VB8BeamseTTpkDA', bytes=35415, created_at=1713116692, filename='chapter_one.txt', object='file', purpose='assistants', status='processed', status_details=None)

In [78]:
client.beta.threads.messages.create(
    thread_id=thread.id, role="user", content="", file_ids=[file.id]
)

Message(id='msg_SG0fOWoqapSjEa40wwTxhlPM', assistant_id=None, completed_at=None, content=[], created_at=1713116695, file_ids=['file-g9IIL5WN6VB8BeamseTTpkDA'], incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_nalDBgi0EVwwpHeofMsaaedN')

In [83]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant_id,
)
run

Run(id='run_q4cLNT7clRSDNLa4PKsoXUjP', assistant_id='asst_8esEH4JjqkuJZaY2GpqZyA4y', cancelled_at=None, completed_at=None, created_at=1713116715, expires_at=1713117315, failed_at=None, file_ids=[], instructions='You help users with their question on the files they upload.', last_error=None, metadata={}, model='gpt-4-1106-preview', object='thread.run', required_action=None, started_at=None, status='queued', thread_id='thread_nalDBgi0EVwwpHeofMsaaedN', tools=[RetrievalTool(type='retrieval')], usage=None, temperature=1.0, max_completion_tokens=None, max_prompt_tokens=None, truncation_strategy={'type': 'auto', 'last_messages': None}, incomplete_details=None, response_format='auto', tool_choice='auto')

In [79]:
def get_run(run_id, thread_id):
    return client.beta.threads.runs.retrieve(
        run_id=run_id,
        thread_id=thread_id,
    )


def send_message(thread_id, content):
    return client.beta.threads.messages.create(
        thread_id=thread_id, role="user", content=content
    )


def get_messages(thread_id):
    messages = client.beta.threads.messages.list(thread_id=thread_id)
    messages = list(messages)
    messages.reverse()
    for message in messages:
        # # print(message.role)
        # print()
        # print(message.content)
        if len(message.content) == 0:
            continue
        print(f"{message.role}: {message.content[0].text.value}")
        for annotation in message.content[0].text.annotations:
            print(f"Source: {annotation.file_citation}")

def get_tool_outputs(run_id, thread_id):
    run = get_run(run_id, thread_id)
    outputs = []
    for action in run.required_action.submit_tool_outputs.tool_calls:
        action_id = action.id
        function = action.function
        print(f"Calling function: {function.name} with arg {function.arguments}")
        outputs.append(
            {
                "output": functions_map[function.name](json.loads(function.arguments)),
                "tool_call_id": action_id,
            }
        )
    return outputs


def submit_tool_outputs(run_id, thread_id):
    outpus = get_tool_outputs(run_id, thread_id)
    return client.beta.threads.runs.submit_tool_outputs(
        run_id=run_id,
        thread_id=thread_id,
        tool_outputs=outpus,
    )

In [88]:
get_run(run.id, thread.id).status

'completed'

In [89]:
get_messages(thread.id)

user: I want you to help me with this file
user: I wnat to know where does Winston lives and how his apartment looks like.
assistant: Winston lives in Victory Mansions, which is located in London, chief city of Airstrip One in Oceania. His apartment is described as having a telescreen that is always transmitting and receiving, meaning that the government could potentially be watching at any time. His apartment contains a tiny kitchen with scarcely any food except for a hunk of dark-coloured bread. The kitchen has a shelf with a bottle of Victory Gin, which gives off a sickly, oily smell. In the living-room, there is a small table that stands to the left of the telescreen. For some reason, the telescreen is in an unusual position on the longer wall opposite the window, and there is a shallow alcove where Winston can sit to remain out of the telescreen's vision range【11†source】.
Source: FileCitation(file_id='file-g9IIL5WN6VB8BeamseTTpkDA', quote="Winston turned round abruptly. He had set

In [80]:
send_message(
    thread.id,
    "I wnat to know where does Winston lives and how his apartment looks like.",
)

Message(id='msg_MLY7qUu9TkfoklT6brJguAbP', assistant_id=None, completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='I wnat to know where does Winston lives and how his apartment looks like.'), type='text')], created_at=1713116706, file_ids=[], incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_nalDBgi0EVwwpHeofMsaaedN')

In [ ]:
# send_message -> get_message -> run -> status 확인 -> get_message